In [1]:
import numpy as np
import cv2

In [3]:
wat = f"D:\DTU-D\4\Fagprojekt\Data\photos\00a53a17-50db-4a3d-be1a-31ffd9537030\image.jpg"

In [5]:
# load the image and convert it to grayscale
image = cv2.imread("D:/DTU-D/4/Fagprojekt/Data/photos/00a53a17-50db-4a3d-be1a-31ffd9537030/image.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# compute the Scharr gradient magnitude representation of the images
# in both the x and y direction
gradX = cv2.Sobel(gray, ddepth = cv2.CV_32F, dx = 1, dy = 0, ksize = -1)
gradY = cv2.Sobel(gray, ddepth = cv2.CV_32F, dx = 0, dy = 1, ksize = -1)
# subtract the y-gradient from the x-gradient
gradient = cv2.subtract(gradX, gradY)
gradient = cv2.convertScaleAbs(gradient)

In [7]:
from PIL import Image
img = Image.fromarray(gradient, 'RGB')
img.show()

ValueError: not enough image data